# Overhead of using a generator

## Plain function calls

In [31]:
def test(a):
    return a

%timeit test(1)

10000000 loops, best of 3: 106 ns per loop


So we get 1E7 function calls per second (a few thousand CPU clock cycles). This is about 100 times slower than what can be achieved in low level languages.
CPython does use the C stack for function calls and spins up a new interpreter loop for each call. However, it uses a custom frame object for the actual Python content.

## Plain list

In [15]:
numbers = list(range(1000))

%timeit sum(numbers)

100000 loops, best of 3: 10.2 µs per loop


## Generator

In [16]:
def generator_func():
    i = 0
    while i < 1000:
        i += 1
        yield i

%timeit sum(generator_func())

10000 loops, best of 3: 120 µs per loop


So while this is ten times slower, we can still manage about 1E7 iterations per second (similar to the number of function calls we can manage).
Under the hood CPython keeps the frame object and reuses it when entering the function again, but a new C stack frame is pushed each time.

What about nesting generators?

In [25]:
def generator_func():
    i = 0
    while i < 1000:
        i += 1
        yield i
        
def generator_wrapper(inner_iterator):
    yield from inner_iterator

%timeit sum(generator_wrapper(generator_wrapper(generator_func())))

1000 loops, best of 3: 194 µs per loop


So while nesting added some overhead, the effect is not dramatic (we still manage about five million iterations per second).

## Plain function calls

In [28]:
def test_func(i):
    return i + 1;

def generator_calling_func():
    i = 0
    while i < 1000:
        i = test_func(i)
        yield i
        
%timeit sum(generator_calling_func())

1000 loops, best of 3: 207 µs per loop


# asyncio overhead

In [32]:
import asyncio
loop = asyncio.get_event_loop()

In [34]:
async def hello_world():
    await asyncio.sleep(1.0)
    print('Hello World!')

%timeit loop.run_until_complete(hello_world())

Hello World!
Hello World!
Hello World!
Hello World!
1 loops, best of 3: 1 s per loop
